## Import packages

In [1]:
cd /Users/zoltan/Dropbox/Python

/Users/zoltan/Dropbox/Python


In [2]:
pwd

u'/Users/zoltan/Dropbox/Python'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import cline_analysis as ca
import pandas as pd
import seaborn as sns
import datetime
import os
from scipy.signal import medfilt
import functools
from scipy.optimize import bisect
from scipy import stats

sns.set_style("whitegrid")
sns.set_style("ticks")
%matplotlib qt
%config InlineBackend.figure_format = 'svg'
plt.matplotlib.rcParams['svg.fonttype'] = 'svgfont' # fonts will be recognized by Adobe Illustrator

## Load data

In [4]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Tarauaca/csv_files/'
fnames,clxs,clys,rbxs,lbxs,rbys,lbys,curvatures,ages,widths,dates = ca.load_data(dirname)

In [5]:
fnames

['Tarauaca_20070729.csv',
 'Tarauaca_20170724.csv',
 'Tarauaca_Jurua_20070729.csv',
 'Tarauaca_Jurua_20170724.csv']

In [6]:
dates

[datetime.datetime(2007, 7, 29, 0, 0),
 datetime.datetime(2017, 7, 24, 0, 0),
 datetime.datetime(2007, 7, 29, 0, 0),
 datetime.datetime(2017, 7, 24, 0, 0)]

## Get migration rate

In [7]:
ts1 = 0 # first timestep
ts2 = 1 # second timestep

d = dates[ts2]-dates[ts1]
years = d.days/365.0

x = np.array(clxs[ts1])
y = np.array(clys[ts1])

x = x[:12741] # get rid of extra length in x and y
y = y[:12741]

xn = np.array(clxs[ts2])
yn = np.array(clys[ts2])

migr_rate, migr_sign, p, q = ca.get_migr_rate(x,y,xn,yn,years,0)

In [15]:
# qc
plt.figure()
plt.plot(x,y)
plt.plot(xn,yn)
plt.axis('equal');

In [12]:
migr_rate = medfilt(savgol_filter(migr_rate,41,3),kernel_size=5) # smoothing
curv,s = ca.compute_curvature(x,y)
curv = medfilt(savgol_filter(curv,71,3),kernel_size=5) # smoothing

In [13]:
# set intervals affected by cu=toffs to NaN - specific to Tarauaca river 
migr_rate[4672:4905] = np.NaN
migr_rate[5461:5699] = np.NaN
migr_rate[6812:6995] = np.NaN

In [14]:
# qc
plt.figure()
plt.plot(migr_rate)

In [8]:
from shapely.geometry import LineString
import geopandas as gpd

def write_shapefile(x,y,epsg,dirname,filename):
    coords=[]
    for i in range(len(x)):
        coords.append((x[i],y[i]))
    ls = LineString(coords)
    ls = gpd.GeoSeries(ls)
    ls.crs = {'init' :'epsg:'+str(epsg)}
    ls.to_file(dirname+filename+'.shp')
    
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Curvature_paper_II/GIS_data/Tarauaca/'
epsg = 32619

year = str(dates[ts1].year)
day = str(dates[ts1].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts1].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(x,y,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts1],lbys[ts1],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts1],rbys[ts1],epsg=epsg,dirname=dirname,filename=filename)

year = str(dates[ts2].year)
day = str(dates[ts2].day)
if len(day)==1:
    day = '0'+day   
month = str(dates[ts2].month)
if len(month)==1:
    month = '0'+month
filename = year+month+day+'_cl'
write_shapefile(xn,yn,epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_lb'
write_shapefile(lbxs[ts2],lbys[ts2],epsg=epsg,dirname=dirname,filename=filename)
filename = year+month+day+'_rb'
write_shapefile(rbxs[ts2],rbys[ts2],epsg=epsg,dirname=dirname,filename=filename)

## Read 'valid' inflection points and corresponding points of zero migration from CSV file

In [16]:
df = pd.read_csv('Tarauaca_LT05_L1TP_004065_20070729_20161112_01_T1_inflection_and_zero_migration_indices.csv')
LZC = np.array(df['index of inflection point'])
LZM = np.array(df['index of zero migration'])

In [17]:
# indices of bends affected by low erodibility and cutoffs (these have been picked manually)
erodibility_inds = [93,97,113,143,147]
cutoff_inds = [47,48,66,67,142,159,160]

## Plot curvature and migration rate series side-by-side

In [19]:
# plot curvature and migration rate along the channel

W = np.nanmean(widths[0])

fig, ax1 = plt.subplots(figsize=(25,4))
plt.tight_layout()

curv_scale = 0.4
migr_scale = 15
y1 = curv_scale
y2 = -3*curv_scale
y3 = 3*migr_scale
y4 = -migr_scale

y5 = -2*curv_scale
y6 = 2*migr_scale

for i in range(0,len(LZC)-1,2):
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,0.85,0.85],edgecolor='k',zorder=0)

deltas = 25.0
ax1.fill_between(s, 0, curv*W)
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_rate, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y2,y1)
ax2.set_ylim(y4,y3)
ax1.set_xlim(0,s[-1])

for i in erodibility_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[1.0,0.85,0.85],edgecolor='k',zorder=0) 
    
for i in cutoff_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,1.0,0.85],edgecolor='k',zorder=0) 
    
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y1,y1,0,y5,y2,y2,y5,0]
        ax1.fill(xcoords,ycoords,color='w') 
        
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y3,y3,y6,0,y4,y4,0,y6]
        ax2.fill(xcoords,ycoords,color='w') 

for i in range(0,len(LZC)-1,2):
    ax1.text(s[LZC[i]],0.5,str(i),fontsize=12)


## Estimate lag between curvature and migration rate

In [23]:
# plot widths and boundary between the two segments
plt.figure()
plt.plot(s,widths[0][:12741])
plt.plot([s[9475],s[9475]],[0,250],'r')

In [24]:
# first segment
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(widths[0][:9475])

94.58626630546298

In [25]:
# second segment
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(widths[0][9475:])

146.80917772320933

In [27]:
np.mean(25.0*(LZM[:140]-LZC[:140]))

443.57142857142856

In [26]:
np.mean(25.0*(LZM[140:]-LZC[140:]))

534.09090909090912

# First segment (Tarauaca A)

## Estimate friction factor Cf

In [35]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 0
i2 = 9475
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [31]:
plt.figure()
plt.plot(migr_rate_t)

In [36]:
migr_rate_t[4672:4905] = 0
migr_rate_t[5461:5699] = 0
migr_rate_t[6812:6995] = 0

In [37]:
# this might take a while to run
kl = 25.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[0][:i2])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
curv_t[4672:4905] = 0
curv_t[5461:5699] = 0
curv_t[6812:6995] = 0

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.003903515625


In [38]:
Cf_opt = 0.003903515625

## Estimate migration rate constant kl

In [42]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
# curv_t, s = ca.compute_curvature(xt,yt)
# curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing
for i in np.arange(10,40):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(10,40),errors);

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [43]:
kl_opt = 31.0 # the error is at minimum for kl = 31.0

In [45]:
444/25.0

17.76

In [44]:
plt.figure()
plt.plot(W*kl_opt*curv_t)
plt.plot(migr_rate_t)

## Plot actual migration rate against nominal migration rate

In [47]:
migr_rate_t[4672:4905] = np.NaN
migr_rate_t[5461:5699] = np.NaN
migr_rate_t[6812:6995] = np.NaN

# kernel density and scatterplot of actual vs. nominal migration rate
w = np.nanmean(widths[0][:9475]) 

curv_nodim = W*curv_t*kl_opt
lag = 18
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 17.0
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--') 
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [48]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.645999757937
0.417315687255
0.0


In [49]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

8803

In [50]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [51]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r')

In [52]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.636577276786
0.40523062932
0.0


In [53]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

12.710110374800637

In [54]:
# plot actual vs. predicted migration rate
max_m = 17.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [55]:
# plot actual vs. predicted migration rate
max_m = 20.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::10],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::10],c='r',s=15)
for i in cutoff_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::10],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::10],c='g',s=15)

# Second segment (Tarauaca B)

## Estimate friction factor Cf

In [56]:
# first we need a continuous channel segment (e.g., no NaNs due to cutoffs)
q=np.array(q)
p=np.array(p)
         
i1 = 9475
i2 = len(x)-1
i1n = p[np.where(q==i1)[0][0]]
i2n = p[np.where(q==i2)[0][0]]

xt = x[i1:i2]
yt = y[i1:i2]
xnt = xn[i1n:i2n]
ynt = yn[i1n:i2n]

plt.figure()
plt.plot(xt,yt)
plt.plot(xnt,ynt)
plt.axis('equal')

migr_rate_t, migr_sign_t, pt, qt = ca.get_migr_rate(xt,yt,xnt,ynt,years,0)

plt.figure()
plt.plot(migr_rate_t)

In [57]:
# this might take a while to run
kl = 25.0 # preliminary kl value (guesstimate)
k = 1
W = np.mean(widths[0][i1:])
D = (W/18.8)**0.7092 # depth in meters (from width)

dx,dy,ds,s = ca.compute_derivatives(xt,yt)
curv_t, s = ca.compute_curvature(xt,yt)
curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

migr_rate_t = medfilt(savgol_filter(migr_rate_t,41,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.004683203125


In [58]:
Cf_opt = 0.004683203125

## Estimate migration rate constant kl

In [59]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
for i in np.arange(10,40):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(10,40),errors);

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [60]:
kl_opt = 25.0 # the error is at minimum for kl = 25.0

In [62]:
534/25.0 # lag

21.36

In [61]:
plt.figure()
plt.plot(W*kl_opt*curv_t)
plt.plot(migr_rate_t)

## Plot actual migration rate against nominal migration rate

In [63]:
# kernel density and scatterplot of actual vs. nominal migration rate
w = np.nanmean(widths[0][i1:]) 

curv_nodim = W*curv_t*kl_opt
lag = 21
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0], migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate_t[lag:][::20],c='k',s=15)
max_x = 15.0
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--') 
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [64]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0],
                                                                      migr_rate_t[lag:][np.isnan(migr_rate_t[lag:])==0])
print r_value
print r_value**2
print p_value

0.806775498977
0.650886705749
0.0


In [65]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate_t[lag:])==0])

3244

In [66]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(xt,yt)
R1 = ca.get_predicted_migr_rate(curv_t,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [67]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate_t)
plt.plot(s,R1,'r')

In [68]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate_t[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate_t)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.789850118104
0.623863209069
0.0


In [69]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

10.223005238794242

In [70]:
# plot actual vs. predicted migration rate
max_m = 15.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [71]:
# plot actual vs. predicted migration rate
max_m = 15.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::5],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::5],c='r',s=15)
for i in cutoff_inds:
    plt.scatter(R1[-i1+LZC[i]:-i1+LZC[i+1]][::5],migr_rate_t[-i1+LZC[i]:-i1+LZC[i+1]][::5],c='g',s=15)